In [3]:
import sqlite3
import pandas as pd
import sys
import os

# Add the project root to the path so we can import from db module
sys.path.append(os.path.dirname(os.getcwd()))

# Import the database client to use the proper connection method
from db.db_client import get_connection, init_schema

# Initialize the database schema first
init_schema()

# Get connection using the proper method
conn = get_connection()

# Show all tables in the database
df_tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Available tables:")
print(df_tables)

# Close the connection
conn.close()
print("Done!")

Available tables:
                  name
0              usersV2
1             licenses
2        subscriptions
3      user_licensesV2
4                roles
5         user_rolesV2
6               groups
7        user_groupsV2
8        azure_devices
9       intune_devices
10      user_devicesV2
11        backup_radar
12            amx_orgs
13         amx_devices
14  amx_device_details
15        amx_packages
Done!


In [ ]:
# Example queries using your database tables
from db.db_client import get_connection, query

# Get connection
conn = get_connection()

# Example 1: Count records in each table
table_counts = []
for table_name in df_tables['name'].values:
    if table_name != 'sqlite_sequence':  # Skip system table
        count_query = f"SELECT COUNT(*) as count FROM {table_name}"
        result = pd.read_sql_query(count_query, conn)
        table_counts.append({'table': table_name, 'count': result['count'].iloc[0]})

counts_df = pd.DataFrame(table_counts)
print("Record counts by table:")
print(counts_df)

# Example 2: Sample data from usersV2 table (if it has data)
try:
    users_sample = pd.read_sql_query("SELECT * FROM usersV2 LIMIT 5", conn)
    print("\nSample users data:")
    print(users_sample)
except Exception as e:
    print(f"\nNo data in usersV2 table yet: {e}")

# Example 3: Sample data from azure_devices table (if it has data)
try:
    devices_sample = pd.read_sql_query("SELECT * FROM azure_devices LIMIT 5", conn)
    print("\nSample Azure devices data:")
    print(devices_sample)
except Exception as e:
    print(f"\nNo data in azure_devices table yet: {e}")

# Close connection
conn.close()


In [4]:
# Device Patch Compliance Query
# Using amx_packages table to analyze patch compliance by tenant and devices

from db.db_client import get_connection

# Get connection
conn = get_connection()

# Query 1: Overall patch compliance by organization (tenant)
print("=== PATCH COMPLIANCE BY ORGANIZATION ===")
compliance_by_org_query = """
SELECT 
    ad.organization_id,
    ao.display_name as org_name,
    COUNT(DISTINCT ad.device_id) as total_devices,
    COUNT(DISTINCT CASE WHEN ap.installed = 1 THEN ad.device_id END) as devices_with_patches,
    COUNT(DISTINCT CASE WHEN ap.installed = 0 AND ap.ignored = 0 THEN ad.device_id END) as devices_needing_patches,
    COUNT(DISTINCT CASE WHEN ap.ignored = 1 THEN ad.device_id END) as devices_with_ignored_patches,
    ROUND(
        (COUNT(DISTINCT CASE WHEN ap.installed = 1 THEN ad.device_id END) * 100.0) / 
        COUNT(DISTINCT ad.device_id), 2
    ) as compliance_percentage
FROM amx_devices ad
JOIN amx_orgs ao ON ad.organization_id = ao.organization_id
LEFT JOIN amx_packages ap ON ad.organization_id = ap.organization_id AND ad.device_id = ap.device_id
GROUP BY ad.organization_id, ao.display_name
ORDER BY compliance_percentage DESC;
"""

# Execute and display results
cursor = conn.cursor()
cursor.execute(compliance_by_org_query)
results = cursor.fetchall()

print("Organization | Total Devices | Patched | Need Patches | Ignored | Compliance %")
print("-" * 80)
for row in results:
    print(f"{row[1][:20]:<20} | {row[2]:<12} | {row[3]:<7} | {row[4]:<12} | {row[5]:<7} | {row[6]:<12}")

print("\n" + "="*80)


=== PATCH COMPLIANCE BY ORGANIZATION ===
Organization | Total Devices | Patched | Need Patches | Ignored | Compliance %
--------------------------------------------------------------------------------
Divergeit            | 109          | 108     | 86           | 2       | 99.08       



In [5]:
# Query 2: Device-level patch compliance details
print("=== DEVICE-LEVEL PATCH COMPLIANCE ===")
device_compliance_query = """
SELECT 
    ad.organization_id,
    ao.display_name as org_name,
    ad.device_id,
    ad.display_name as device_name,
    ad.connected,
    ad.is_compliant,
    COUNT(ap.package_version_id) as total_packages,
    COUNT(CASE WHEN ap.installed = 1 THEN 1 END) as installed_packages,
    COUNT(CASE WHEN ap.installed = 0 AND ap.ignored = 0 THEN 1 END) as pending_packages,
    COUNT(CASE WHEN ap.ignored = 1 THEN 1 END) as ignored_packages,
    COUNT(CASE WHEN ap.severity = 'Critical' AND ap.installed = 0 AND ap.ignored = 0 THEN 1 END) as critical_pending,
    COUNT(CASE WHEN ap.severity = 'High' AND ap.installed = 0 AND ap.ignored = 0 THEN 1 END) as high_pending,
    ROUND(
        (COUNT(CASE WHEN ap.installed = 1 THEN 1 END) * 100.0) / 
        NULLIF(COUNT(ap.package_version_id), 0), 2
    ) as patch_compliance_percentage
FROM amx_devices ad
JOIN amx_orgs ao ON ad.organization_id = ao.organization_id
LEFT JOIN amx_packages ap ON ad.organization_id = ap.organization_id AND ad.device_id = ap.device_id
GROUP BY ad.organization_id, ad.device_id, ad.display_name, ad.connected, ad.is_compliant
ORDER BY patch_compliance_percentage ASC, critical_pending DESC, high_pending DESC
LIMIT 20;
"""

cursor.execute(device_compliance_query)
device_results = cursor.fetchall()

print("Org | Device Name | Connected | Compliant | Total | Installed | Pending | Critical | High | Compliance %")
print("-" * 100)
for row in device_results:
    org_name = row[1][:8] if row[1] else "N/A"
    device_name = row[3][:15] if row[3] else "N/A"
    connected = "Yes" if row[4] else "No"
    compliant = "Yes" if row[5] else "No"
    print(f"{org_name:<8} | {device_name:<15} | {connected:<9} | {compliant:<9} | {row[6]:<5} | {row[7]:<9} | {row[8]:<7} | {row[9]:<8} | {row[10]:<4} | {row[11]:<12}")

print("\n" + "="*100)

# Additional Query 2A: Summary of devices with pending patches vs no pending patches
print("\n=== DEVICES WITH PENDING PATCHES vs NO PENDING PATCHES ===")
pending_summary_query = """
SELECT 
    ao.display_name as org_name,
    COUNT(DISTINCT ad.device_id) as total_devices,
    COUNT(DISTINCT CASE WHEN pending_count.pending_packages > 0 THEN ad.device_id END) as devices_with_pending_patches,
    COUNT(DISTINCT CASE WHEN pending_count.pending_packages = 0 OR pending_count.pending_packages IS NULL THEN ad.device_id END) as devices_with_no_pending_patches,
    ROUND(
        (COUNT(DISTINCT CASE WHEN pending_count.pending_packages > 0 THEN ad.device_id END) * 100.0) / 
        COUNT(DISTINCT ad.device_id), 2
    ) as percentage_with_pending_patches
FROM amx_devices ad
JOIN amx_orgs ao ON ad.organization_id = ao.organization_id
LEFT JOIN (
    SELECT 
        organization_id, 
        device_id, 
        COUNT(CASE WHEN installed = 0 AND ignored = 0 THEN 1 END) as pending_packages
    FROM amx_packages 
    GROUP BY organization_id, device_id
) pending_count ON ad.organization_id = pending_count.organization_id AND ad.device_id = pending_count.device_id
GROUP BY ad.organization_id, ao.display_name
ORDER BY percentage_with_pending_patches DESC;
"""

cursor.execute(pending_summary_query)
pending_summary_results = cursor.fetchall()

print("Organization | Total Devices | With Pending | No Pending | % With Pending")
print("-" * 75)
for row in pending_summary_results:
    print(f"{row[0][:15]:<15} | {row[1]:<12} | {row[2]:<11} | {row[3]:<9} | {row[4]:<13}")

print("\n" + "="*75)

# Additional Query 2B: Package count per device
print("\n=== PACKAGE COUNT PER DEVICE ===")
package_count_query = """
SELECT 
    ad.organization_id,
    ao.display_name as org_name,
    ad.device_id,
    ad.display_name as device_name,
    ad.connected,
    COUNT(ap.package_version_id) as total_packages,
    COUNT(CASE WHEN ap.installed = 1 THEN 1 END) as installed_packages,
    COUNT(CASE WHEN ap.installed = 0 AND ap.ignored = 0 THEN 1 END) as pending_packages,
    COUNT(CASE WHEN ap.ignored = 1 THEN 1 END) as ignored_packages,
    COUNT(CASE WHEN ap.severity = 'Critical' THEN 1 END) as critical_packages,
    COUNT(CASE WHEN ap.severity = 'High' THEN 1 END) as high_packages,
    COUNT(CASE WHEN ap.severity = 'Medium' THEN 1 END) as medium_packages,
    COUNT(CASE WHEN ap.severity = 'Low' THEN 1 END) as low_packages
FROM amx_devices ad
JOIN amx_orgs ao ON ad.organization_id = ao.organization_id
LEFT JOIN amx_packages ap ON ad.organization_id = ap.organization_id AND ad.device_id = ap.device_id
GROUP BY ad.organization_id, ad.device_id, ad.display_name, ad.connected
ORDER BY total_packages DESC, pending_packages DESC
LIMIT 25;
"""

cursor.execute(package_count_query)
package_count_results = cursor.fetchall()

print("Org | Device Name | Connected | Total | Installed | Pending | Ignored | Critical | High | Medium | Low")
print("-" * 95)
for row in package_count_results:
    org_name = row[1][:8] if row[1] else "N/A"
    device_name = row[3][:15] if row[3] else "N/A"
    connected = "Yes" if row[4] else "No"
    print(f"{org_name:<8} | {device_name:<15} | {connected:<9} | {row[5]:<5} | {row[6]:<9} | {row[7]:<7} | {row[8]:<7} | {row[9]:<8} | {row[10]:<4} | {row[11]:<6} | {row[12]:<3}")

print("\n" + "="*95)


=== DEVICE-LEVEL PATCH COMPLIANCE ===
Org | Device Name | Connected | Compliant | Total | Installed | Pending | Critical | High | Compliance %
----------------------------------------------------------------------------------------------------
Divergei | DESKTOP-6KPH3FJ | No        | No        | 0     | 0         | 0       | 0        | 0    | 0           
Divergei | GERALDLIPAN89C9 | No        | No        | 220   | 190       | 30      | 0        | 0    | 0           
Divergei | win11-divergeit | No        | No        | 189   | 174       | 15      | 0        | 0    | 0           
Divergei | PK-LAPTOP       | No        | No        | 124   | 116       | 8       | 0        | 0    | 0           
Divergei | Mac.attlocal.ne | No        | No        | 310   | 291       | 19      | 0        | 0    | 0           
Divergei | Desktop-JK_Home | No        | No        | 460   | 432       | 28      | 0        | 0    | 0           
Divergei | MyDell          | No        | No        | 742   | 697       |

In [ ]:
# Query 3: Critical security patches requiring immediate attention
print("=== CRITICAL SECURITY PATCHES REQUIRING ATTENTION ===")
critical_patches_query = """
SELECT 
    ad.organization_id,
    ao.display_name as org_name,
    ad.device_id,
    ad.display_name as device_name,
    ad.connected,
    ap.display_name as package_name,
    ap.version as package_version,
    ap.severity,
    ap.cve_score,
    ap.cves,
    ap.scheduled_at,
    CASE 
        WHEN ap.scheduled_at IS NOT NULL THEN 'Scheduled'
        ELSE 'Not Scheduled'
    END as patch_status
FROM amx_devices ad
JOIN amx_orgs ao ON ad.organization_id = ao.organization_id
JOIN amx_packages ap ON ad.organization_id = ap.organization_id AND ad.device_id = ap.device_id
WHERE ap.installed = 0 
    AND ap.ignored = 0 
    AND ap.severity IN ('Critical', 'High')
    AND ad.connected = 1
ORDER BY 
    CASE ap.severity 
        WHEN 'Critical' THEN 1 
        WHEN 'High' THEN 2 
        ELSE 3 
    END,
    ap.cve_score DESC,
    ao.display_name,
    ad.display_name
LIMIT 25;
"""

cursor.execute(critical_patches_query)
critical_results = cursor.fetchall()

print("Org | Device | Package | Version | Severity | CVE Score | Status")
print("-" * 80)
for row in critical_results:
    org_name = row[1][:8] if row[1] else "N/A"
    device_name = row[3][:12] if row[3] else "N/A"
    package_name = row[5][:15] if row[5] else "N/A"
    version = row[6][:10] if row[6] else "N/A"
    severity = row[7] if row[7] else "N/A"
    cve_score = row[8] if row[8] else "N/A"
    status = row[11] if row[11] else "N/A"
    
    print(f"{org_name:<8} | {device_name:<12} | {package_name:<15} | {version:<10} | {severity:<8} | {cve_score:<9} | {status}")

# Close connection
conn.close()
print("\n" + "="*80)
print("Query completed successfully!")
